# Fine-tunning BirdNET (AKA shallow classifier)

Para esta sesion vamos a cargar el ambiente virtual que creamos para la primera practica ademas del modelo "Zoo"

In [ ]:
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
import random
from glob import glob
import sklearn

from tqdm.autonotebook import tqdm
from sklearn.metrics import average_precision_score, roc_auc_score
from pathlib import Path

#set up plotting
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for large visuals
%config InlineBackend.figure_format = 'retina'

# opensoundscape transfer learning tools
from opensoundscape.ml.shallow_classifier import MLPClassifier, quick_fit, fit_classifier_on_embeddings

In [ ]:
# Define las "Seeds"
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

# Preparación  del set de datos

In [ ]:
# Luego de mover la carpeta descargada, define la ruta:
dataset_path = Path("./rana_sierrae_2022/")

# Dado que usaremos el modelo "BirdNET", es necesario estandarizar la duracion de todos los clips a 3 segundos
from opensoundscape.annotations import BoxedAnnotations

audio_and_raven_files = pd.read_csv(f"{dataset_path}/audio_and_raven_files.csv")

# Actualiza la lista donde estan guardados los audios y anotaciones
audio_and_raven_files["audio"] = audio_and_raven_files["audio"].apply(
    lambda x: f"{dataset_path}/{x}"
)
audio_and_raven_files["raven"] = audio_and_raven_files["raven"].apply(
    lambda x: f"{dataset_path}/{x}"
)

# Crea las tabla con anotaciones (similar a la primera sesion practica)
annotations = BoxedAnnotations.from_raven_files(
    raven_files=audio_and_raven_files["raven"],
    audio_files=audio_and_raven_files["audio"],
    annotation_column="annotation",
)

# Ahora generemos etiquetas para clips cada 3 segundos, incluyendo cualquier etiqueta que se sobrelape por al menos 0.2s
labels = annotations.clip_labels(clip_duration=3, min_label_overlap=0.2)

In [ ]:
# Ahora inspeccionemos las etiquetas
labels.sum()

In [ ]:
# Dividamos las etiquetas para crear sets de entrenamiento y evaluacion (No recomendado)
labels_train, labels_val = sklearn.model_selection.train_test_split(labels[["A"]])

# Arquitectura del clasificador

In [ ]:
import bioacoustics_model_zoo as bmz

# Primero, carguemos BirdNET de Zoo
birdnet = bmz.BirdNET()

In [ ]:
# Ahora generemos embeddings a para cada uno de los elementos en nuestros datasets
emb_train = birdnet.embed(labels_train, return_dfs=False, batch_size=128, num_workers=0)
emb_val = birdnet.embed(labels_val, return_dfs=False, batch_size=128, num_workers=0)

In [ ]:
# Para entrenar el clasificador sobre la clase "A" debemos reemplazar el output de la fully-connected layer con una capa de 1-output para clase "A" 
classes = ["A"]
birdnet.change_classes(classes)

# fit the classification head with embeddings and labels
birdnet.network.fit(emb_train, labels_train.values, emb_val, labels_val.values)

In [ ]:
# Hagamos predicciones pasando los embeddings a traves del clasificador
preds = birdnet.network(torch.tensor(emb_val)).detach()
# Calculemos el AUC-ROC
roc_auc_score(labels_val.values, preds, average=None)

# Veamos los histogramas con los scores
preds = preds.detach().numpy()
plt.hist(preds[labels_val == True], bins=20, alpha=0.5, label="positives")
plt.hist(preds[labels_val == False], bins=20, alpha=0.5, label="negatives")
plt.legend()